## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,68.29,100,75,3.44,light rain
1,1,High Level,CA,58.5169,-117.1360,60.91,82,75,2.30,broken clouds
2,2,Moron,AR,-34.6534,-58.6198,49.75,68,10,4.41,clear sky
3,3,Saint-Philippe,RE,-21.3585,55.7679,63.99,82,36,12.24,light rain
4,4,Saint George,US,37.1041,-113.5841,102.79,7,0,12.66,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip?"))

max_temp=float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df=city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>=min_temp)]

clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kavieng,PG,-2.5744,150.7967,85.37,66,43,13.18,scattered clouds
9,9,Zabid,YE,14.1951,43.3152,86.70,56,100,5.75,overcast clouds
11,11,Avarua,CK,-21.2078,-159.7750,78.85,65,40,16.11,scattered clouds
15,15,Kapaa,US,22.0752,-159.3190,85.98,78,40,16.11,scattered clouds
17,17,Vaini,TO,-21.2000,-175.2000,82.56,78,40,9.22,scattered clouds
20,20,Namatanai,PG,-3.6667,152.4333,84.81,66,50,10.58,scattered clouds
23,23,Callaway,US,30.1530,-85.5699,81.00,82,40,4.61,scattered clouds
25,25,Hit,IQ,33.6416,42.8251,83.23,17,0,11.23,clear sky
27,27,Jeremie,HT,18.6500,-74.1167,79.95,83,94,0.72,overcast clouds
30,30,Hilo,US,19.7297,-155.0900,80.28,82,100,11.50,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                208
City                   208
Country                208
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kavieng,PG,-2.5744,150.7967,85.37,66,43,13.18,scattered clouds
9,9,Zabid,YE,14.1951,43.3152,86.70,56,100,5.75,overcast clouds
11,11,Avarua,CK,-21.2078,-159.7750,78.85,65,40,16.11,scattered clouds
15,15,Kapaa,US,22.0752,-159.3190,85.98,78,40,16.11,scattered clouds
17,17,Vaini,TO,-21.2000,-175.2000,82.56,78,40,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
670,670,Chabahar,IR,25.2919,60.6430,85.53,74,93,15.95,overcast clouds
672,672,Cedar City,US,37.6775,-113.0619,86.97,10,0,10.36,clear sky
675,675,Umm Kaddadah,SD,13.6017,26.6876,83.93,42,91,6.69,overcast clouds
677,677,Macrohon,PH,10.0797,124.9431,83.70,70,68,4.41,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kavieng,PG,85.37,scattered clouds,-2.5744,150.7967,
9,Zabid,YE,86.70,overcast clouds,14.1951,43.3152,
11,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
15,Kapaa,US,85.98,scattered clouds,22.0752,-159.3190,
17,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,
20,Namatanai,PG,84.81,scattered clouds,-3.6667,152.4333,
23,Callaway,US,81.00,scattered clouds,30.1530,-85.5699,
25,Hit,IQ,83.23,clear sky,33.6416,42.8251,
27,Jeremie,HT,79.95,overcast clouds,18.6500,-74.1167,
30,Hilo,US,80.28,overcast clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    
    lat=row["Lat"]
    
    lng=row["Lng"]
    
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    
    except (IndexError):
        
        print("Hotel not found ...skipping.")
        
hotel_df
        

Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.
Hotel not found ...skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kavieng,PG,85.37,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
9,Zabid,YE,86.70,overcast clouds,14.1951,43.3152,مغرب التريبه
11,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
15,Kapaa,US,85.98,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
17,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
670,Chabahar,IR,85.53,overcast clouds,25.2919,60.6430,هتل گدروشیا
672,Cedar City,US,86.97,clear sky,37.6775,-113.0619,Hampton Inn Cedar City
675,Umm Kaddadah,SD,83.93,overcast clouds,13.6017,26.6876,
677,Macrohon,PH,83.70,broken clouds,10.0797,124.9431,Banahaw Creek


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.replace(r'^\s*$', np.nan, regex=True)

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kavieng,PG,85.37,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
9,Zabid,YE,86.70,overcast clouds,14.1951,43.3152,مغرب التريبه
11,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
15,Kapaa,US,85.98,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
17,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
670,Chabahar,IR,85.53,overcast clouds,25.2919,60.6430,هتل گدروشیا
672,Cedar City,US,86.97,clear sky,37.6775,-113.0619,Hampton Inn Cedar City
675,Umm Kaddadah,SD,83.93,overcast clouds,13.6017,26.6876,NaN
677,Macrohon,PH,83.70,broken clouds,10.0797,124.9431,Banahaw Creek


In [10]:
clean_hotel_df.dropna()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kavieng,PG,85.37,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
9,Zabid,YE,86.70,overcast clouds,14.1951,43.3152,مغرب التريبه
11,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
15,Kapaa,US,85.98,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
17,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
666,Galveston,US,85.05,scattered clouds,29.3669,-94.9669,Crystal Suites
670,Chabahar,IR,85.53,overcast clouds,25.2919,60.6430,هتل گدروشیا
672,Cedar City,US,86.97,clear sky,37.6775,-113.0619,Hampton Inn Cedar City
677,Macrohon,PH,83.70,broken clouds,10.0797,124.9431,Banahaw Creek


In [11]:
# 8a. Create the output File (CSV)

output_data_file="Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Description</dt><dd>{Current Description}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))